# Decompose SST gradients into large-scale and mesoscale.

In [1]:
# Import libraries
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs

import pylab as plt
import numpy as np
# Inline plotting
%matplotlib inline

In [2]:
from dask.distributed import Client

In [3]:
c = Client()
c

Client Scheduler: tcp://127.0.0.1:54522 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [ ]:
import astropy 

In [9]:
dataset_SST_grad = xr.open_dataset('../datasets/SST_grads_timeseries.nc',chunks={'lat':10,'lon':10})

In [11]:
dataset_SST_grad.SST_grad.attrs

{'units': '$^\\circ C$',
 'name': 'SST',
 'long_name': 'Sea Surface Temperature',
 'valid_min': -3.0396215e-12,
 'valid_max': 9.19988e-05,
 'valid_range': array([-3.0396215e-12,  9.1998802e-05], dtype=float32)}